In [3]:
import geopandas as gpd
import pandas as pd
from shapely.ops import snap
from tqdm import tqdm

In [4]:
# df: gpd.GeoDataFrame = gpd.read_file("http://localhost:3055/bus/routes/geojson")\
# df: gpd.GeoDataFrame = gpd.read_file("https://trainstat.us/api/bus/routes/geojson")
df: gpd.GeoDataFrame = gpd.read_file("routes.geojson")

In [5]:
# df_cl: gpd.GeoDataFrame = gpd.read_file("CSCL.geojson")
df_cl = gpd.read_file("https://data.cityofnewyork.us/api/geospatial/exjm-f27b?method=export&format=GeoJSON")

In [6]:
df = df.to_crs(2263)
df_cl = df_cl.to_crs(2263)

In [7]:
df_filtered = df[df['id'].isin(["Q10", "QM18"])]
df_filtered

,color,id,long_name,short_name,shuttle,geometry
180,#FAA61A,Q10,Kew Gardens - Jfk Airport,Q10,False,"MULTILINESTRING ((1031458.419 199342.548, 1031..."
269,#00933C,QM18,South Ozone Park - Midtown Express,QM18,False,"MULTILINESTRING ((990063.326 211112.618, 99001..."


In [8]:
df_exploded = df_filtered.explode().reset_index(drop=True)
intersections_info = []
df_exploded['overlapping'] = False
# print(df_exploded)

with tqdm(total=len(df_exploded)**2) as pb:
    for i, row in df_exploded.iterrows():
        for j, row2 in df_exploded.drop(i).iterrows():
            intersection = row['geometry'].intersection(row2['geometry'])
            # print(intersection)
            # Check if intersection is not empty
            if not intersection.is_empty:
                # df_exploded.iloc[i]['offset'] = 5
                # df_exploded.at[i, 'offset'] = 10
                df_exploded.at[i, 'overlapping'] = True

            #     intersections_info.append((i, True))
            # else:
            #     intersections_info.append((i, False))
            pb.update(1)
# print(df_exploded['offset'].unique())
# df_exploded[df_exploded['offset'] == 10]
df_exploded.to_crs(4326).to_file('offsets.geojson')

/tmp/ipykernel_57075/2133425740.py:1: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  df_exploded = df_filtered.explode().reset_index(drop=True)
 98%|█████████▊| 3080/3136 [00:00<00:00, 20372.13it/s]


In [26]:
geoms = []
with tqdm(total=len(df_filtered)) as pb:
    for i, row in df_filtered.iterrows():
        # Drop current index so we don't check identical routes
        
        for i, row2 in df_filtered.drop(i).iterrows():
            intersections = row['geometry'].intersection(row2['geometry'])
            geoms.append(intersections)
df_filtered.geometry = geoms
df_filtered.to_crs(4326).to_file("intersections.geojson")

  0%|          | 0/2 [00:00<?, ?it/s]
c:\Users\JMoskal\AppData\Local\ESRI\conda\envs\arcgispro-py3-scripts\Lib\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,color,id,long_name,short_name,shuttle,geometry
0,#00AEEF,B1,Bay Ridge - Manhattan Beach,B1,False,"LINESTRING (1000981.137 149932.399, 1000972.78..."
0,#00AEEF,B1,Bay Ridge - Manhattan Beach,B1,False,"LINESTRING (976324.357 165848.968, 976285.507 ..."
0,#00AEEF,B1,Bay Ridge - Manhattan Beach,B1,False,"LINESTRING (977934.235 164941.324, 977948.113 ..."
0,#00AEEF,B1,Bay Ridge - Manhattan Beach,B1,False,"LINESTRING (978142.406 164777.324, 978358.904 ..."
0,#00AEEF,B1,Bay Ridge - Manhattan Beach,B1,False,"LINESTRING (978758.607 164288.98, 979163.87 16..."
...,...,...,...,...,...,...
355,#00933C,X68,Floral Park - Midtown Manhattan Express,X68,False,"LINESTRING (990760.568 214923.71, 990832.563 2..."
355,#00933C,X68,Floral Park - Midtown Manhattan Express,X68,False,"LINESTRING (991516.492 216293.816, 991582.943 ..."
355,#00933C,X68,Floral Park - Midtown Manhattan Express,X68,False,"LINESTRING (992807.374 216512.836, 992912.666 ..."
355,#00933C,X68,Floral Park - Midtown Manhattan Express,X68,False,"LINESTRING (991337.441 212807.101, 991401.133 ..."
